In [ ]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from statsmodels.stats.multitest import multipletests


import scanpy as sc

In [ ]:
f = '/u/home/f/f7xiesnm/v1_multiome/multiome_cell_sex_assignment_saumya.csv'
df_sex = pd.read_csv(f)
df_sex

In [ ]:
f = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/superdupermegaRNA_hasraw_multiome_l23.h5ad'
adata = sc.read(f)
adata

In [ ]:
adata.obs = adata.obs.join(df_sex.set_index('cell'))

In [ ]:
adatasub = adata[(adata.obs['Age']=='P17') & (adata.obs['Subclass']=='L2/3')]
adatasub

In [ ]:
# normalize
mat = np.log2(np.array(adatasub.raw.X.todense())/adatasub.obs['total_counts'].values.reshape(-1,1)*1e4+1)
mat


In [ ]:
df = pd.DataFrame(mat, columns=adatasub.var.index.values, index=adatasub.obs.index)
df = df.join(adatasub.obs[['sex', 'Sample']])
df = df.dropna()
df

In [ ]:
gene = 'Xist'

# Fit LMM: random intercept model
model = smf.mixedlm(f"{gene} ~ sex", df, groups="Sample")
result = model.fit()
print(result.summary())

In [ ]:
pval = result.pvalues['sex[T.male]']
pval

In [ ]:
# rejs, qvals = multipletests(pvals, alpha=0.05, method='fdr_bh')